In [20]:
import os
import time
import pickle
from project.control import *
from project.ftp_utils import *
from project.coordinate_convert import *

img = 'realsense.jpg'

changeState(0)

while True:
    # 1 -> 2
    # take picture and send it to ftp server
    print('wait for state 1')
    waitState(1)
    
    pipeline = init_color_camera()
    take_picture(pipeline, img)
#     pipeline.stop()
    print('picture taken')
    send_ftp(img)

    changeState(2)
    print('proceed to stage 2')

    # 4 -> 5
    # receive detection result and execute arm
    print('wait for state 4')
    waitState(4)

    recv_ftp('result.txt')
    with open('result.txt', 'r') as f:
        data = f.readlines()
    
    data = data[:-1]
    if data !=  None:
        max_score, max_record = 0, None
        for record in data:
            if float(record.split('|')[4]) > max_score:
                max_score = float(record.split('|')[4])
                max_record = record[:-2]

        [y_min, x_min, y_max, x_max, _, class_id] = [float(i) for i in max_record.split('|')]
        y = y_min + (y_max - y_min)/ 2
        x = x_min + (x_max - x_min)/ 2

        # load coordinate
        with open('project/coordinate.pkl', 'rb') as f:
            coor = pickle.load(f)

        # execute arm
        pos = convert_coor(coor, (x, y))
        print(pos)
        execute( '%.2f %0.2f 0 %d' % (pos[0], pos[1], 2 if int(class_id) != 1 else 1) )

    changeState(0)

wait for state 1
picture taken
proceed to stage 2
wait for state 4


KeyboardInterrupt: 

In [15]:
# pos = convert_coor(coor, (288.0952453613281, 135.84540367126465))

max_record.split('|')

['140.83195209503174',
 '372.12528228759766',
 '212.52352237701416',
 '449.3029022216797',
 '0.9100836',
 '1.0',
 '']

In [57]:
from project.ftp_utils import *

changeState(0)
checkState()

'0'